#### Import Dependencies

In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, SparseVectorParams, Document, Prefetch, FusionQuery
from qdrant_client import models

import pandas as pd
import openai
import fastembed

/home/k/AI-Engineering-Bootcamp/01-ai-engineering-bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
qdrant_client = QdrantClient(url="http://localhost:6333")

#### Create Qdrant Collection for Hybrid Search

In [3]:
qdrant_client.create_collection( 
    collection_name="Amazon-items-collection-01-hybrid-search",
    vectors_config={"text-embedding-3-small": VectorParams(size=1536, distance=Distance.COSINE)}, 
    sparse_vectors_config={"bm25": SparseVectorParams(modifier=models.Modifier.IDF)}
)

True

In [4]:
qdrant_client.create_payload_index(  
    collection_name="Amazon-items-collection-01-hybrid-search",   
    field_name="parent_asin", 
    field_schema=PayloadSchemaType.KEYWORD, 
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

#### Embedding functions

In [20]:
def get_embedding(text, model="text-embedding-3-small"): 
    response = openai.embeddings.create(  
        input=[text], 
        model=model 
    )
    return response.data[0].embedding

In [5]:
def get_embeddings_batch(text_list, model="text-embedding-3-small", batch_size=100):
    
    if len(text_list) <= batch_size:
        response = openai.embeddings.create(input=text_list, model=model)
        return [embedding.embedding for embedding in response.data]
    
    all_embeddings = []
    counter = 1
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i + batch_size]
        response = openai.embeddings.create(input=batch, model=model)
        all_embeddings.extend([embedding.embedding for embedding in response.data])
        print(f"Processed {counter * batch_size} of {len(text_list)}")
        counter += 1
    
    return all_embeddings

#### Process and Embed Amazon Items Data

In [6]:
df_items = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", lines=True)

In [7]:
def preprocess_description(row):
    return f"{row['title']} {' '.join(row['features'])}"

In [8]:
def extract_first_large_image(row):
    return row["images"][0].get("large", "")

In [9]:
df_items["description"] = df_items.apply(preprocess_description, axis=1)
df_items["image"] = df_items.apply(extract_first_large_image, axis=1)

In [10]:
df_sample = df_items.sample(500, random_state=42)

In [11]:
data_to_embed = df_sample[["description", "image", "rating_number", "price", "average_rating", "parent_asin"]].to_dict(orient="records")

In [12]:
data_to_embed

[{'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Palm Rejection]- 

In [13]:
text_to_embed = [data["description"] for data in data_to_embed]

In [14]:
text_to_embed

['KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Palm Rejection]- Rest your palm o

In [15]:
embeddings = get_embeddings_batch(text_to_embed)

Processed 100 of 500
Processed 200 of 500
Processed 300 of 500
Processed 400 of 500
Processed 500 of 500


In [16]:
pointstructs = [] 
i = 1 
for embedding, data in zip(embeddings, data_to_embed): 
    pointstructs.append(
        PointStruct( 
            id=i, 
            vector={
                "text-embedding-3-small": embedding, 
                "bm25": Document( 
                    text=data["description"],  
                    model="qdrant/bm25"
                )
            }, 
            payload=data 
        )
    )
    i += 1 

In [ ]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-01-hybrid-search",
    points=pointstructs
)

#### Hybrid Retrieval

In [21]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid-search",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    retrieved_context_ratings = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [22]:
results = retrieve_data("Can I get some tablet?", qdrant_client, k=20)

In [23]:
results

{'retrieved_context_ids': ['B0B8NVNQKX',
  'B09LH466KZ',
  'B0BR33XH8D',
  'B0BN7WWH63',
  'B0C78B1BTB',
  'B0BL2CZSHT',
  'B0BFGXRMJN',
  'B08CGM6Y1J',
  'B0C3XYD574',
  'B0C772GZRF',
  'B0BTPG4XHH',
  'B0BVZ47N6N',
  'B0BC78N85F',
  'B0BRXZDBXZ',
  'B0BMVR2MVC',
  'B0BBPVCFW7',
  'B0BP1YXF3R',
  'B0BS15TRJ3',
  'B09PYKX76X',
  'B0BSGDCVND'],
 'retrieved_context': ['COOPERS 7 inch Kids Tablet Android 11 Tablet for Kids, 2GB RAM + 32GB ROM Toddler Tablet PC for Children, IPS Touch Screen, Dual Camera, Dual Speaker, WiFi Computer Tablet, Light Blue ✿【Good Kids tablet】This 7 inch tablet for kids with silm body and lightweight, it is easy to hold by children. Also the special design can protect the tablet well when dropping. ✿【Parental Control】Toddlers Tablet with parent mode can add or block apps. Set screen time limits. This tablet come with iwawa app. kids can get access to fun and educational games and videos. ✿【Powerful Tablets】Equipmented with quad core CPU, Android 11.0 System, 32G